<a href="https://colab.research.google.com/github/yukinaga/hopfield_boltzmann/blob/main/section_2/03_noise_reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ホップフィールドネットワークによるノイズ除去
ホップフィールドネットワークを用いてシンプルな二値画像（白黒画像）をノイズ除去します。  
画像パターンを学習させ、ノイズの入ったバージョンを入力して元の画像に復元する方法を示します。

## コード：ホップフィールドネットワークを使った画像のノイズ除去
以下では、**ホップフィールドネットワークを使った画像のノイズ除去** の例を紹介します。  
このコードでは、ホップフィールドネットワークを用いてシンプルな二値画像（白黒画像）をノイズ除去します。  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class HopfieldNetwork:
    def __init__(self, num_neurons):
        self.num_neurons = num_neurons
        self.weights = np.zeros((num_neurons, num_neurons))

    def train(self, patterns):
        """パターンを学習し、重み行列を構築する"""
        for p in patterns:
            p = p.reshape(self.num_neurons, 1)  # パターンを縦ベクトルに変換
            self.weights += np.dot(p, p.T)      # 外積で重み行列を更新
        np.fill_diagonal(self.weights, 0)       # 対角要素を0に設定

    def recall(self, pattern, steps=10):
        """パターンを再現し、ノイズを除去する"""
        result = pattern.copy()
        for _ in range(steps):
            result = np.sign(np.dot(self.weights, result))
        return result

def add_noise(pattern, noise_level=0.3):
    """ノイズを追加してパターンを変更する"""
    noisy_pattern = pattern.copy()
    num_noisy_pixels = int(noise_level * len(pattern))
    noise_indices = np.random.choice(len(pattern), num_noisy_pixels, replace=False)
    noisy_pattern[noise_indices] *= -1  # ピクセルを反転（1→-1、-1→1）
    return noisy_pattern

def plot_image(vector, title, image_shape=(8, 8)):
    """1次元配列を画像としてプロット"""
    plt.imshow(vector.reshape(image_shape), cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

# 8x8 のシンプルな二値画像パターン (1: 白, -1: 黒)
pattern1 = np.array([
    1, 1, 1, -1, -1, 1, 1, 1,
    1, -1, -1, -1, -1, -1, 1, 1,
    1, -1, 1, 1, 1, -1, 1, 1,
    1, -1, 1, -1, 1, -1, 1, 1,
    1, -1, 1, 1, 1, -1, 1, 1,
    1, -1, -1, -1, -1, -1, 1, 1,
    1, 1, 1, -1, -1, 1, 1, 1,
    1, 1, 1, 1, 1, 1, 1, 1
])

pattern2 = np.array([
    -1, -1, 1, 1, 1, 1, -1, -1,
    -1, 1, -1, -1, -1, -1, 1, -1,
    1, -1, -1, -1, -1, -1, -1, 1,
    1, -1, -1, -1, -1, -1, -1, 1,
    1, -1, -1, -1, -1, -1, -1, 1,
    1, -1, -1, -1, -1, -1, -1, 1,
    -1, 1, -1, -1, -1, -1, 1, -1,
    -1, -1, 1, 1, 1, 1, -1, -1
])

# ホップフィールドネットワークを初期化してパターンを学習
network = HopfieldNetwork(num_neurons=64)
network.train([pattern1, pattern2])

# 学習に使用したパターンを表示
plot_image(pattern1, "Training Pattern 1")
plot_image(pattern2, "Training Pattern 2")

# ノイズを追加したパターン
noisy_pattern = add_noise(pattern1, noise_level=0.3)
plot_image(noisy_pattern, "Noisy Pattern")

# ノイズ除去（再現）
recalled_pattern = network.recall(noisy_pattern)
plot_image(recalled_pattern, "Recalled Pattern (Denoised)")

### コードの解説

1. **`HopfieldNetwork` クラス**:
   - `train` メソッド:
     - 学習パターン（`pattern1` と `pattern2`）を用いて重み行列を更新。
   - `recall` メソッド:
     - ノイズの入ったパターンを修正し、学習したパターンに収束させます。

2. **`add_noise` 関数**:
   - 指定された割合のピクセルをランダムに反転してノイズを追加します。

3. **`plot_image` 関数**:
   - 1次元のパターンを指定された形状（8×8）に変換し、画像として表示します。

4. **学習と再現の流れ**:
   - 学習パターン (`pattern1`, `pattern2`) をホップフィールドネットワークに学習させます。
   - 学習パターンを表示します（`Training Pattern 1` と `Training Pattern 2`）。
   - `pattern1` にノイズを追加し、ノイズの入ったパターンを表示します（`Noisy Pattern`）。
   - ノイズの入ったパターンをネットワークに入力し、復元結果を表示します（`Recalled Pattern (Denoised)`）。

---

### 実行結果の例

- **Training Pattern 1**:
  - 学習に使用された `pattern1` の画像。
  
- **Training Pattern 2**:
  - 学習に使用された `pattern2` の画像。

- **Noisy Pattern**:
  - `pattern1` にノイズを加えた画像。

- **Recalled Pattern (Denoised)**:
  - ホップフィールドネットワークが `Noisy Pattern` を元の `pattern1` に復元した画像。

---

### まとめ

- ホップフィールドネットワークは、学習したパターンを記憶し、ノイズの入った入力から元のパターンを復元する能力を持っています。
- 今回の例では、8×8の単純な二値パターンを学習し、ノイズ除去を行いました。
- さらに大きな画像や複雑なパターンでも同様の手法を適用することで、ノイズ除去やパターン認識が可能です。